In [ ]:
import pandas as pd
from plotly import offline as pyo
from plotly import graph_objects as go
import numpy as np
import os
from scipy import ndimage as ndi
from jw_utils import parse_gff as pgf
from jw_utils import parse_fasta as pf
from jw_utils import parse_gbk as pgb
from jw_utils import file_utils as fu
from Bio import SeqIO
import bisect
import pysam
from transcript_calling import tc_functions as tf
from jw_utils import genome_utils as gu
path_to_gff = '../references/FERM_BP3421.gff'

In [ ]:
path_to_gff = '../references/FERM_BP3421.gff'
path_to_strGTF_neg = './merge_all_negStrand.gtf'
path_to_valcount_dir = '../results/analysis/value_counts'
path_to_annot_file = '../data/references/Reference_FERM_BP3421.gbk'
path_to_fa_genomes = '../data/references/concat_references.fa'

## Code below from first attempt to find Eustaquio TSS

In [11]:
sam_align_obj = pysam.AlignmentFile(path_to_bam_mergedAlignments_neg, 'rb')
bf1_pilup = sam_align_obj.pileup(contig='BF000000.1')


## Description of initial steps
#### raw data description './data/...'
working directory: ('~/Dropbox/Trestle_projects/Eustaquio_lab/RNAseq_dash_app')
1) 5 chromosomes/plasmids: ('BF000000.1', 'BF000000.2', 'BF000000.3', 'BF000000.4', 'BF000000.5') './data/references' 
    - combined these files into one fasta ( './data/references/concat_references.fa' )
2) 12 paired end fastq files (6 R1 +  6 R2) ('../Illumina_RNA_Reads/fastq_files')
    - aligned with bowtie2 with our nextflow pipeline (see the multiqc report) using standard settings (../new_alignments/Bowtie_aligned')
    - extracted just 5' (rna) end of the paired end segment
        -5’ end of reads with flags 147 or 163 faced same direction as predicted RNAs



In [ ]:
#### All paths to data

### Make a example of clean data and difference kernal and plot it

y0 = np.full(50, 0)
y1 = np.full(50, 1)
y = np.concatenate([y0,y1])
x = np.array(range(len(y)))
t = ndi.convolve(y, [1,0,-1], mode='reflect')
fig = pu.quick_line(x =x, y = t, name='convolved w diff kernal' )
data = pu.quick_line(x =x, y = y, name='clean data'  )['data'][0]
fig = fig.add_trace(data)
pyo.plot(fig)

### First processing steps
1) Make dataframe of summed reads at each genomic position from all experiments
2) Make transcription start site TssFinder object
3) Find all tss windows and return df dictionary with reads for each gene. 100 nt before start codon up to start codon
4) Make figure dictionary containing plotly figs for reads in the tss window and for the diff kernal-convolved reads

In [ ]:
import processing as pr
df = pr.get_read_sums(path_to_valcount_dir,'BF000000.1')
tf_sums = gp.TssFinder(path_to_annot_file, val_counts_df=df, contig_name='BF000000.1')
tss_windows = tf_sums.get_region_hits(read_density_cut=0.2)
fig_dict= pr.make_fig_dict(path_to_annot_file,contig_name='BF000000.1', kernal_type='diff',plot_kernal=False, read_density_cut=0.2)

print(gb_dict['BF000000.1']['tmp_003403'].qualifiers['product'])
f = fig_dict['tmp_003403']



fig_dict= pr.make_fig_dict(path_to_annot_file,contig_name='BF000000.1', kernal_type='diff',plot_kernal=False, read_density_cut=0.2)
genes= list(fig_dict.keys()) 
plus_genes_tss = list(set(genes).intersection(set(tf_sums.plus_strand_genes)))  #get all genes on the +strand that have windows
gb_dict = pgb.build_genbank_dict(path_to_annot_file)



def get_tss(tss_window):
    ser = ndi.convolve(tss_window,[1,0,-1])
    df = pd.DataFrame({gene:tss_window, 'diff_kernal':ser}, index = tss_window.index)
    filt1 = df.loc[:,'diff_kernal'] > 0
    filt2 = df.loc[:,gene] > 0
    df = df.loc[filt1,:] 
    df = df.loc[filt2,:]
    d = {}
    pre = {}
    post = {}
    window_size = 20
    for position in df.index:
        ave_reads = tss_window.sum()/tss_window.shape[0]
        pre_window_reads = tss_window.loc[position-window_size:position].sum()
        post_window_reads = tss_window.loc[position:position+window_size].sum()
        if post_window_reads/window_size < ave_reads/2:
            post_window_reads=0
        #print(f'position: {position} post_window_reads: {post_window_reads}')
        if pre_window_reads==0:
            pre_window_reads=pre_window_reads+0.000000001
        #print(f'position: {position} pre_window_reads: {pre_window_reads}\n')
        d[position]=(post_window_reads-pre_window_reads)/pre_window_reads
        pre[position] = pre_window_reads
        post[position] = post_window_reads
    df_hypoth = pd.DataFrame().from_dict(d, orient='index')
    df_hypoth.columns = ['enrichment']
    df_hypoth['post'] = list(post.values())
    df_hypoth['pre'] = list(pre.values())
    return df_hypoth.sort_values('enrichment', ascending=False)

# df_dict = {}
# for gene in plus_genes_tss:
#     df_dict[gene] = get_tss(tss_windows[gene])


###pick a gene
# gene = plus_genes_tss[8]
for i in range(50):
    gene = plus_genes_tss[i]
    print(gene)
    print(gb_dict['BF000000.1'][gene].location.strand)
    print(gb_dict['BF000000.1'][gene].qualifiers['product'])
    fig = fig_dict[gene]
    df = get_tss(tss_windows[gene])
    for i,pos in enumerate(df.index):
        if i<1:
            fig.add_vline(x=pos,line_width=1, line_dash='dash')
    fig.write_html(f'./figs/{gene}_putative_tss.html')

a = fig_dict[gene]
#pf.get_seq_region(path_to_fa_genomes,'BF000000.1',18757,18857)


name='Histogram of read density in tss window'
ser = pd.Series(dens_dict.values())
#
trace = pu.plot_bar_with_outliers(ser, name=name, end = 20)
pyo.plot(go.Figure(data=trace))

### Sandbox for developing kernal to find steps

import numpy as np
from matplotlib import pyplot as plt


d = '''594.          568.55555556  577.22222222  624.55555556  546.66666667
552.88888889  575.55555556  592.33333333  528.88888889  576.11111111
625.          574.22222222  556.33333333  567.66666667  576.66666667
591.66666667  566.33333333  567.33333333  547.44444444  631.11111111
555.66666667  548.66666667  579.44444444  546.88888889  597.55555556
519.88888889  582.33333333  618.88888889  574.55555556  547.44444444
593.11111111  565.66666667  544.66666667  562.66666667  554.11111111
543.88888889  602.33333333  609.77777778  550.55555556  561.88888889
719.33333333  784.44444444  711.22222222  843.66666667  691.33333333
690.11111111  684.33333333  749.11111111  759.11111111  653.33333333
817.11111111  705.22222222  689.44444444  712.33333333  659.
683.88888889  713.          740.44444444  692.22222222  677.33333333
681.44444444  640.          717.55555556  717.88888889  769.22222222
690.88888889  786.          774.66666667  799.44444444  743.44444444
789.88888889  673.66666667  685.66666667  709.88888889  645.55555556
846.11111111  792.77777778  702.22222222  749.44444444  678.55555556
707.55555556  665.77777778  643.55555556  671.44444444  795.66666667
627.22222222  684.55555556  708.44444444  829.66666667  719.        '''

dary = np.array([*map(float, d.split())])

dary -= np.average(dary)

step = np.hstack((np.ones(len(dary)), -1*np.ones(len(dary))))

dary_step = np.convolve(dary, step, mode='valid')

# get the peak of the convolution, its index

step_indx = np.argmax(dary_step)  # yes, cleaner than np.where(dary_step == dary_step.max())[0][0]

# plots

plt.plot(dary)

plt.plot(dary_step/10)

plt.plot((step_indx, step_indx), (dary_step[step_indx]/10, 0), 'r')